# LSTM

In [1]:
import io
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

import tensorflow_datasets as tfds

2023-10-31 12:56:11.400636: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/chiraj/iWork/TensorFlow/tensorflow-cert/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Subword tokens for IMDB Dataset

In [2]:
imdb_subwords, info_subwords = tfds.load("imdb_reviews/subwords8k", with_info=True, as_supervised=True)

In [3]:
tokenizer = info_subwords.features['text'].encoder

## Prepare Dataset

###### Note: To make the training go faster in this lab, you will increase the batch size that Laurence used in the lecture. In particular, you will use 256 and this takes roughly a minute to train per epoch

In [4]:
BUFFER_SIZE = 10000
BATCH_SIZE = 256

# Get the train and test splits
train_data, test_data = imdb_subwords['train'], imdb_subwords['test'], 

# Shuffle the training data
train_dataset = train_data.shuffle(BUFFER_SIZE)

# Batch and pad the datasets to the maximum length of the sequences
train_dataset = train_dataset.padded_batch(BATCH_SIZE)
test_dataset = test_data.padded_batch(BATCH_SIZE)

## Create Model

- We will simply swap the Flatten or GlobalAveragePooling1D from before with an LSTM layer
- Moreover, we will nest it inside a Biderectional layer so the passing of the sequence information goes both forwards and backwards
- These additional computations will naturally make the training go slower than the models
- We should take this into account when using RNNs in our own applications

In [5]:
embedding_dim = 64
lstm_dim = 64
dense_dim = 64

# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size, embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_dim)),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          523840    
                                                                 
 bidirectional (Bidirection  (None, 128)               66048     
 al)                                                             
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 598209 (2.28 MB)
Trainable params: 598209 (2.28 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [6]:
# Set the training parameters
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## Train Model

In [7]:
NUM_EPOCHS = 10

history = model.fit(train_dataset, epochs=NUM_EPOCHS, validation_data=test_dataset)

Epoch 1/10
98/98 [==============================] - 561s 6s/step - loss: 0.6025 - accuracy: 0.6669 - val_loss: 0.4433 - val_accuracy: 0.8043
Epoch 2/10
98/98 [==============================] - 548s 6s/step - loss: 0.3525 - accuracy: 0.8562 - val_loss: 0.4456 - val_accuracy: 0.7972
Epoch 3/10
98/98 [==============================] - 551s 6s/step - loss: 0.2713 - accuracy: 0.8985 - val_loss: 0.4119 - val_accuracy: 0.8407
Epoch 4/10
98/98 [==============================] - 548s 6s/step - loss: 0.2144 - accuracy: 0.9229 - val_loss: 0.4052 - val_accuracy: 0.8468
Epoch 5/10
98/98 [==============================] - 541s 6s/step - loss: 0.1858 - accuracy: 0.9324 - val_loss: 0.4116 - val_accuracy: 0.8503
Epoch 6/10
25/98 [======>.......................] - ETA: 11:47 - loss: 0.1657 - accuracy: 0.9420

KeyboardInterrupt: 

## Visualize Training

In [8]:
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

# Plot the accuracy and results 
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

NameError: name 'history' is not defined